In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage,fcluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, adjusted_rand_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import svm
from scipy.stats import pearsonr, spearmanr


<ipython-input-1-b3d3bdcee481>:7: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

<ipython-input-1-b3d3bdcee481>:8: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [2]:
import pandas as pd

# Read the CSV files
commodity = pd.read_csv('commodity_f.csv',sep = ',')

In [3]:
commodity

,Country,commodity,flow,trade_usd,quantity_name,quantity,category,Carbon Footprint
0,Afghanistan,"Sheep, live",Export,6088.000,Number of items,51.00,01_live_animals,0.18
1,Afghanistan,"Goats, live",Export,3958.000,Number of items,53.00,01_live_animals,0.18
2,Albania,"Bovine animals, live, except pure-bred breeding",Import,2414533.000,Number of items,6853.00,01_live_animals,0.87
3,Albania,"Swine, live except pure-bred breeding > 50 kg",Import,7648728.875,Number of items,96040.00,01_live_animals,0.87
4,Albania,"Fowls, live domestic < 185 grams",Import,2671732.000,Number of items,4895775.75,01_live_animals,0.87
...,...,...,...,...,...,...,...,...
37403,Madagascar,"Seaweeds and other algae,",Export,1014203.000,Weight in kilograms,1687717.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37404,Madagascar,Vegetable products nes for human consumption,Import,4137.000,Weight in kilograms,1369.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37405,Madagascar,"Cereal straw & husks, unprepared",Export,11.000,Weight in kilograms,4.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37406,Madagascar,Lucerne (alfalfa) meal and pellets,Import,1720.000,Weight in kilograms,2000.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07


In [4]:
commodity.dtypes

Country              object
commodity            object
flow                 object
trade_usd           float64
quantity_name        object
quantity            float64
category             object
Carbon Footprint    float64
dtype: object

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
commodity['flow'] = encoder.fit_transform(commodity['flow'])
commodity['quantity_name'] = encoder.fit_transform(commodity['quantity_name'])

In [6]:
# Séparer les features et la variable cible
X = commodity[['flow', 'trade_usd', 'quantity_name', 'quantity', 'Carbon Footprint']]
y = commodity['category']

In [7]:
commodity.dtypes

Country              object
commodity            object
flow                  int32
trade_usd           float64
quantity_name         int32
quantity            float64
category             object
Carbon Footprint    float64
dtype: object

In [8]:
commodity.dtypes

Country              object
commodity            object
flow                  int32
trade_usd           float64
quantity_name         int32
quantity            float64
category             object
Carbon Footprint    float64
dtype: object

In [9]:
commodity= commodity.drop(columns=['Country', 'commodity'])

In [11]:

from sklearn.metrics import accuracy_score
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Créer un modèle d'arbre de décision
decision_tree = DecisionTreeClassifier()

# Entraîner le modèle sur les données d'entraînement
decision_tree.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred_tree = decision_tree.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred_tree)
print("Précision : %.2f%%" % (accuracy * 100.0))

Précision : 23.30%


In [12]:
com_svm=commodity

In [13]:
import seaborn as sb
import numpy as np

features =[ 'flow', 'trade_usd', 'quantity_name', 'quantity', 'Carbon Footprint']

def impute(com_svm, feature):
    q1 = np.percentile(com_svm[feature] , 25)
    q3 = np.percentile(com_svm[feature] , 75)
    iqr = q3 - q1 
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    com_svm.loc[com_svm[feature] < lower, feature] = lower
    com_svm.loc[com_svm[feature] > upper, feature] = upper

for feature in features:
    impute(com_svm, feature)

In [14]:
# Séparer la variable cible des variables d'entrée
X1 = com_svm.drop('category', axis=1)
y1 = com_svm['category']

In [15]:
X1

,flow,trade_usd,quantity_name,quantity,Carbon Footprint
0,0.0,6088.000,2.0,51.00,0.18
1,0.0,3958.000,2.0,53.00,0.18
2,1.0,2414533.000,2.0,6853.00,0.87
3,1.0,7648728.875,2.0,96040.00,0.87
4,1.0,2671732.000,2.0,4895775.75,0.87
...,...,...,...,...,...
37403,0.0,1014203.000,2.0,1687717.00,0.07
37404,1.0,4137.000,2.0,1369.00,0.07
37405,0.0,11.000,2.0,4.00,0.07
37406,1.0,1720.000,2.0,2000.00,0.07


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Créer un objet RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
random_forest.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_forest = random_forest.predict(X_test)

# Calculer l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred_forest)
print("Accuracy : {:.2f}%".format(accuracy*100))

Accuracy : 26.60%


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Créer un objet KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# Entraîner le modèle
knn.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_knn = knn.predict(X_test)

# Calculer l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy : {:.2f}%".format(accuracy*100))

C:\Users\zeffz\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



Accuracy : 18.99%


In [ ]:
y

In [18]:
com_svm.dtypes

flow                float64
trade_usd           float64
quantity_name       float64
quantity            float64
category             object
Carbon Footprint    float64
dtype: object

In [30]:
# Supposons que vous avez une nouvelle observation représentée par les features suivants :
new_observation = [[0.0, 6088.000, 2.0,51.00,0.18]]

# Faire une prédiction avec le modèle de l'arbre de décision entraîné
predicted_categorie = knn.predict(new_observation)

# Afficher la prédiction
print("La prédiction du modèle de l'arbre de décision pour la nouvelle observation est:", predicted_categorie) 

La prédiction du modèle de l'arbre de décision pour la nouvelle observation est: ['01_live_animals']


C:\Users\zeffz\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



In [25]:
import pickle

# Save the trained model to a file
filename = 'commodity-categorie.pkl'
with open(filename, 'wb') as file:
    
    pickle.dump(random_forest, file)